In [146]:
X_train[:2]

,Unnamed: 0,Family,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,"Age_(0.0, 7.9]","Age_(7.9, 14.45]","Age_(14.45, 31.28]",...,Cabin_T,Embarked_C,Embarked_Q,Embarked_S,"Fare_(0, 10]","Fare_(10, 20]","Fare_(20, 30]","Fare_(30, 100]",Alone_0,Alone_1
0,NaN,0.1,0,0,1,0,1,0,0,1,...,0,0,0,1,1,0,0,0,1,0
1,NaN,0.1,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,1,1,0


In [148]:
X_test[:2]

,Unnamed: 0,Family,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,"Age_(0.0, 7.9]","Age_(7.9, 14.45]","Age_(14.45, 31.28]",...,Cabin_T,Embarked_C,Embarked_Q,Embarked_S,"Fare_(0, 10]","Fare_(10, 20]","Fare_(20, 30]","Fare_(30, 100]",Alone_0,Alone_1
0,601.0,0.2,0,0,1,0,1,0,1,0,...,0,0,0,1,1,0,0,0,1,0
1,604.0,0.0,0,0,1,1,0,0,0,1,...,0,0,0,1,1,0,0,0,0,1


In [149]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('my_test.csv') #@!
train_len = len(train_data)

df = pd.concat([train_data, test_data])
df['Title'] = df['Name'].str.extract('([A-Za-z]+)\.').fillna('NA')
df['Age'] = df.groupby(['Title'])['Age'].transform(lambda g: g.fillna(g.median()))
df['Fare'] = df['Fare'].fillna(df['Fare'].median())
df['Fare'] = df.groupby('Ticket')['Fare'].transform(lambda x: x / len(x))
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])
df['Family'] = df['SibSp'] + df['Parch']
df['Alone'] = (df['SibSp'] + df['Parch'] == 0).astype(int)
df['Cabin'] = df['Cabin'].fillna('NA')
df = df.drop(['PassengerId','Name','Title','Ticket'], axis=1)

df['Fare'] = pd.cut(df['Fare'], bins=[0,10,20,30,100])
df['Age'] = pd.cut(df['Age'], bins=[0,7.90,14.45,31.28,120])
df = pd.get_dummies(df, columns=['Pclass','Sex','Age','SibSp','Parch','Cabin','Embarked','Fare','Alone'])

y_train = df[:train_len]['Survived']
y_test = df[train_len:]['Survived'] #@!
X_train = df[:train_len].drop(['Survived'], axis=1)
X_test = df[train_len:].drop(['Survived'], axis=1)

scaler, scaled = MinMaxScaler(), ['Family']
X_train[scaled] = scaler.fit_transform(X_train[scaled])
X_test[scaled] = scaler.transform(X_test[scaled])

m = LogisticRegression()
m.fit(X_train, y_train)

out = pd.DataFrame()
out['PassengerId'] = test_data.PassengerId
out['Survived'] = m.predict(X_test)
out.to_csv('my_submission.csv', index=False)

# cross_val_score(m, X_train, y_train, scoring='accuracy', cv=3).mean()
accuracy_score(y_test, m.predict(X_test)) #@!

0.7914572864321608

In [115]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

def extract_features(data):   
    X = data[['Pclass','Sex','Age','SibSp','Parch','Cabin','Embarked','Name','Fare','Ticket']].copy()
    X['Title'] = X['Name'].str.extract('([A-Za-z]+)\.').fillna('NA')
    X['Title'] = X['Title'].replace({'Mlle':'Miss','Major':'Mr','Col':'Mr','Sir':'Mr','Don':'Mr',
       'Mme':'Miss','Lady':'Mrs','Jonkheer':'Mr','Capt':'Mr','Countess':'Mrs','Ms':'Miss','Dona':'Mrs'})
    X['Age'] = X.groupby(['Title'])['Age'].transform(lambda g: g.fillna(g.median()))
    X['Age'] = pd.cut(X['Age'], bins=[0,7.90,14.45,31.28,120])
    X['Embarked'] = X['Embarked'].fillna(X['Embarked'].mode()[0])
    X['Cabin'] = X['Cabin'].fillna('NA')
    X['Fare'] = X['Fare'].fillna(X['Fare'].median())
    X['Fare'] = X.groupby('Ticket')['Fare'].transform(lambda x: x / len(x))
    X['Fare'] = pd.cut(X['Fare'], bins=[0,10,20,30,100])
    X['Alone'] = (X['SibSp'] + X['Parch'] == 0)
    X = pd.get_dummies(X, columns=['Pclass','Sex','Age','SibSp','Parch','Cabin','Embarked','Fare','Alone'])
    return X.drop(['Name','Title','Ticket'], axis=1)

def scale_features(X_train, X_test):
    scaler, features = MinMaxScaler(), ['Fare']
    X_train[features] = scaler.fit_transform(X_train[features])
    X_test[features] = scaler.transform(X_test[features])
    return X_train, X_test

def align_features(X_train, X_test):
    return X_train.align(X_test, join='outer', axis=1, fill_value=0)

def extract_target(data):
    return data['Survived'].copy()

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

y_train = extract_target(train_data)
X_train = extract_features(train_data)
X_test = extract_features(test_data)
# X_train, X_test = scale_features(X_train, X_test)
X_train, X_test = align_features(X_train, X_test)

m = LogisticRegression()
m.fit(X_train, y_train)

# m = KNeighborsClassifier(n_neighbors=3)
# m.fit(X_train, y_train)

out = pd.DataFrame()
out['PassengerId'] = test_data.PassengerId
out['Survived'] = m.predict(X_test)
out.to_csv('my_submission.csv', index=False)

cross_val_score(m, X_train, y_train, scoring='accuracy', cv=3).mean()
# accuracy_score(y_test, lr.predict(X_test))

0.7934904601571269